In [1]:
!pip install yfinance

In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
symbol = input("Enter stock symbol (e.g., RELIANCE, TCS, INFY): ").upper()
exchange = input("Enter exchange (NS for NSE, BO for BSE): ").upper()
ticker = f"{symbol}.{exchange}"

start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = datetime.today().strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_date, end=end_date)

Enter stock symbol (e.g., RELIANCE, TCS, INFY):  TCS
Enter exchange (NS for NSE, BO for BSE):  NS
Enter start date (YYYY-MM-DD):  2025-01-01


C:\Users\abali\AppData\Local\Temp\ipykernel_27652\915661245.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [4]:
stock = yf.Ticker(ticker)

# Fetch fundamentals
eps = stock.info.get("trailingEps", None)            # Earnings per Share
pe_ratio = stock.info.get("trailingPE", None)        # Price to Earnings ratio
roe = stock.info.get("returnOnEquity", None)         # Return on Equity (fraction, e.g., 0.18 = 18%)
de_ratio = stock.info.get("debtToEquity", None)      # Debt-to-Equity ratio

print("\nFundamental Metrics:")
print(f"EPS: {eps}")
print(f"P/E Ratio: {pe_ratio}")
print(f"ROE: {roe}")
print(f"Debt-to-Equity: {de_ratio}")


Fundamental Metrics:
EPS: 136.08
P/E Ratio: 23.199589
ROE: 0.51589
Debt-to-Equity: 9.714


In [5]:
# --- Moving Averages ---
data["MA50"] = data["Close"].rolling(50).mean()
data["MA200"] = data["Close"].rolling(200).mean()

# --- RSI Calculation ---
delta = data["Close"].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()

rs = gain / loss
data["RSI"] = 100 - (100 / (1 + rs))

print("\nLatest Technical Indicators:")
print(f"50-day MA: {data['MA50'].iloc[-1]}")
print(f"200-day MA: {data['MA200'].iloc[-1]}")
print(f"RSI: {data['RSI'].iloc[-1]}")


Latest Technical Indicators:
50-day MA: 3215.3523193359374
200-day MA: nan
RSI: 63.896325095688894


In [6]:
def evaluate_stock(eps, pe_ratio, roe, de_ratio, ma50, ma200, rsi):
    score = 0

    # Fundamentals
    if eps and eps > 0: score += 1
    if pe_ratio and pe_ratio < 20: score += 1
    if roe and roe > 0.15: score += 1
    if de_ratio and de_ratio < 1: score += 1

    # Technicals
    if ma50 > ma200: score += 1
    if rsi and rsi < 70 and rsi > 30: score += 1

    # Final decision
    if score >= 5:
        return " Strong Buy"
    elif score >= 3:
        return " Buy / Hold"
    else:
        return " Avoid / Weak"

In [7]:
result = evaluate_stock(eps, pe_ratio, roe, de_ratio, data["MA50"].iloc[-1], data["MA200"].iloc[-1], data["RSI"].iloc[-1])
print("Final Recommendation:", result)

Final Recommendation:  Buy / Hold
